In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/Users/Aaron-MAC/Development/Kaggle/Santander")
import pandas as pd
import numpy as np
from modules.pre_process import *
from modules.get_data import *
import matplotlib.pyplot as plt
import cPickle as pickle
import xgboost as xgb
%matplotlib inline

In [30]:
train_df = pd.read_csv('train_feature.csv')
test_df = pd.read_csv('test_feature.csv')
target_df = pd.read_csv('train_target.csv')

In [31]:
for col in test_df.columns:
    if 'pais_residencia' in col:
        test_df[col] = test_df[col]/100.
        train_df[col] = train_df[col]/100.
    elif 'age' in col:
        test_df[col] = test_df[col]/5.
        train_df[col] = train_df[col]/5.
    elif 'f_antiguedad' in col:
        test_df[col] = test_df[col]/10.
        train_df[col] = train_df[col]/10.
    elif 'f_canal_entrada' in col:
        test_df[col] = test_df[col]/100.
        train_df[col] = train_df[col]/100.
    elif 'f_nomprov' in col:
        test_df[col] = test_df[col]/50.
        train_df[col] = train_df[col]/50.

In [32]:
sel = (target_df.iloc[:, 1:]>0).sum(1)>0
y_train = target_df[sel]
X_train = train_df[sel]

In [33]:
X_train = X_train.fillna(-1.).iloc[:, 2:].astype(float)
y_train = y_train.fillna(-1.).iloc[:, 1:].astype(int)

In [34]:
test_code_list = list(test_df['ncodpers'])
X_test = test_df.fillna(-1.).iloc[:, 2:].astype(float)

In [35]:
mask_1 = 1-np.array(test_df.loc[:, ["f_"+x for x in products]].values)
X_train = X_train.values
y_train = y_train.values

In [40]:
print mask_1.shape
print X_test.shape
print X_train.shape
print y_train.shape

(929615, 24)
(929615, 253)
(45691, 253)
(45691,)


In [39]:
arr = []
for i in range(24):
    sel = y_train==i
    
    if len(X_train[sel])>0:
        arr.append(X_train[sel].mean(0))
feature_sel = np.array(arr).std(0)


X_train = X_train[:, feature_sel>0.1]
X_test = X_test[:, feature_sel>0.1]

In [37]:
X = []
Y = []
for x, y in zip(X_train, y_train):
    for idx, item in enumerate(y):
        if item>0:# and idx in [2,4,6,12,17,18,21,22,23]:
            X.append(x)
            Y.append(idx)

X_train = np.array(X)
y_train = np.array(Y)

In [38]:
X_test = np.array(X_test)

In [236]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(verbose=True, n_estimators=50)

In [29]:
X_train.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [41]:
def runXGB(train_X, train_y, seed_val=0):
    param = {}
    param['objective'] = 'multi:softprob'
    #param['eta'] = 0.05
    #param['max_depth'] = 3
    param['silent'] = 0
    param['num_class'] = 24
    param['eval_metric'] = "mlogloss"
    #param['min_child_weight'] = 1
    #param['subsample'] = 0.7
    #param['colsample_bytree'] = 0.7
    #param['seed'] = seed_val
    num_rounds = 200

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    model = xgb.train(plst, xgtrain, num_rounds)	
    return model



model = runXGB(X_train, y_train)
predict = model.predict(xgb.DMatrix(X_test))

In [220]:
predict = model.predict(xgb.DMatrix(np.array(X_test)))

In [ ]:
predict = model.predict_proba(X_test)
print model.classes_
pred = np.zeros((len(predict), 24))
for idx, i in enumerate(model.classes_):
    pred[:,i] = predict[:,idx]
predict = pred
#predict = np.hstack([np.zeros((len(predict), 2)), predict])

In [239]:
predict = boost.predict_proba(X_test)
print model.classes_
pred = np.zeros((len(predict), 24))
for idx, i in enumerate(model.classes_):
    pred[:,i] = predict[:,idx]
predict = pred

AttributeError: 'GradientBoostingClassifier' object has no attribute 'classes_'

In [241]:
predict = np.hstack([np.zeros((len(predict), 2)), predict])

In [43]:
mask_2 = np.ones((len(predict), 24))
mask_2[:, [0,1,3,5,7,8,9,10,11,13,14,15,16,19,20]] = 0.01

predict = predict*mask_1#*mask_2

In [42]:
predict

array([[  3.57598910e-05,   3.57598910e-05,   4.20569631e-05, ...,
          6.32390082e-02,   9.82042402e-03,   3.09516132e-01],
       [  4.55296686e-05,   4.55296686e-05,   5.97558537e-05, ...,
          5.72467335e-02,   6.09631389e-02,   5.08993804e-01],
       [  2.32013626e-05,   2.32013626e-05,   3.70447087e-05, ...,
          9.94575620e-02,   1.12454161e-01,   5.15340090e-01],
       ..., 
       [  2.74186514e-05,   2.74186514e-05,   4.87424550e-05, ...,
          3.25812697e-02,   2.79685464e-02,   4.37661916e-01],
       [  1.70270396e-05,   1.70270396e-05,   7.85066950e-05, ...,
          1.91464815e-02,   1.34884547e-02,   7.02158868e-01],
       [  4.03022977e-06,   4.03022977e-06,   9.64788139e-01, ...,
          6.10219395e-05,   1.20936020e-03,   7.07702944e-03]], dtype=float32)

In [44]:
f = open("outputs/feature_expand_10_sub.csv", 'w')
f.write("ncodpers,added_products\n")

for idx,r in enumerate(predict):
    order = [i[0] for i in sorted(enumerate(r), key=lambda x:x[1], reverse=True)]
    select = order[:min(7,len(order))]
    f.write(str(test_code_list[idx])+","+(" ".join([products[s] for s in select]))+'\n')
f.close()